# Introduction

In this tutorial, we will go through an example to update a preexisting model. This might be useful when you come across additional data that you would want to consider, without having to train a model from scratch.

The main abstraction that Lightwood offers for this is the `BaseMixer.partial_fit()` method. To call it, you need to pass new training data and a held-out dev subset for internal mixer usage (e.g. early stopping). If you are using an aggregate ensemble, it's likely you will want to do this for every single mixer. The convienient `PredictorInterface.adjust()` does this automatically for you.


# Initial model training

First, let's train a Lightwood predictor for the `concrete strength` dataset:

In [1]:
from lightwood.api.high_level import ProblemDefinition, json_ai_from_problem, predictor_from_json_ai
import pandas as pd

INFO:lightwood-2278:No torchvision detected, image helpers not supported.
INFO:lightwood-2278:No torchvision/pillow detected, image encoder not supported


In [2]:
# Load data
df = pd.read_csv('https://raw.githubusercontent.com/mindsdb/lightwood/staging/tests/data/concrete_strength.csv')

df = df.sample(frac=1, random_state=1)
train_df = df[:int(0.1*len(df))]
update_df = df[int(0.1*len(df)):int(0.8*len(df))]
test_df = df[int(0.8*len(df)):]

print(f'Train dataframe shape: {train_df.shape}')
print(f'Update dataframe shape: {update_df.shape}')
print(f'Test dataframe shape: {test_df.shape}')

Train dataframe shape: (103, 10)
Update dataframe shape: (721, 10)
Test dataframe shape: (206, 10)


Note that we have three different data splits.

We will use the `training` split for the initial model training. As you can see, it's only a 20% of the total data we have. The `update` split will be used as training data to adjust/update our model. Finally, the held out `test` set will give us a rough idea of the impact our updating procedure has on the model's predictive capabilities.

In [3]:
# Define predictive task and predictor
target = 'concrete_strength'
pdef = ProblemDefinition.from_dict({'target': target, 'time_aim': 200})
jai = json_ai_from_problem(df, pdef)

# We will keep the architecture simple: a single neural mixer, and a `BestOf` ensemble:
jai.model = {
    "module": "BestOf",
    "args": {
        "args": "$pred_args",
        "accuracy_functions": "$accuracy_functions",
        "submodels": [{
            "module": "Neural",
            "args": {
                "fit_on_dev": False,
                "stop_after": "$problem_definition.seconds_per_mixer",
                "search_hyperparameters": False,
            }
        }]
    }
}

# Build and train the predictor
predictor = predictor_from_json_ai(jai)
predictor.learn(train_df)

INFO:lightwood-2278:Dropping features: []
INFO:lightwood-2278:Analyzing a sample of 979
INFO:lightwood-2278:from a total population of 1030, this is equivalent to 95.0% of your data.
INFO:lightwood-2278:Infering type for: id
INFO:lightwood-2278:Column id has data type integer
INFO:lightwood-2278:Infering type for: cement
INFO:lightwood-2278:Column cement has data type float
INFO:lightwood-2278:Infering type for: slag
INFO:lightwood-2278:Column slag has data type float
INFO:lightwood-2278:Infering type for: flyAsh
INFO:lightwood-2278:Column flyAsh has data type float
INFO:lightwood-2278:Infering type for: water
INFO:lightwood-2278:Column water has data type float
INFO:lightwood-2278:Infering type for: superPlasticizer
INFO:lightwood-2278:Column superPlasticizer has data type float
INFO:lightwood-2278:Infering type for: coarseAggregate
INFO:lightwood-2278:Column coarseAggregate has data type float
INFO:lightwood-2278:Infering type for: fineAggregate
INFO:lightwood-2278:Column fineAggrega

In [4]:
# Train and get predictions for the held out test set
predictions = predictor.predict(test_df)
predictions

INFO:lightwood-2278:Dropping features: []
INFO:lightwood-2278:Cleaning the data
INFO:lightwood-2278:Dropping features: []
DEBUG:lightwood-2278: `preprocess` runtime: 0.01 seconds
INFO:lightwood-2278:Featurizing the data
DEBUG:lightwood-2278: `featurize` runtime: 0.0 seconds
INFO:lightwood-2278:The block ICP is now running its explain() method
INFO:lightwood-2278:The block AccStats is now running its explain() method
INFO:lightwood-2278:AccStats.explain() has not been implemented, no modifications will be done to the data insights.
INFO:lightwood-2278:The block ConfStats is now running its explain() method
INFO:lightwood-2278:ConfStats.explain() has not been implemented, no modifications will be done to the data insights.
DEBUG:lightwood-2278: `predict` runtime: 1.8 seconds


,original_index,prediction,confidence,lower,upper
0,0,49.177926,0.9991,0.0,99.741836
1,1,18.238433,0.9991,0.0,68.802343
2,2,22.289931,0.9991,0.0,72.853840
3,3,20.362166,0.9991,0.0,70.926075
4,4,38.186172,0.9991,0.0,88.750082
...,...,...,...,...,...
201,201,47.799281,0.9991,0.0,98.363190
202,202,41.190282,0.9991,0.0,91.754192
203,203,37.798300,0.9991,0.0,88.362210
204,204,29.786588,0.9991,0.0,80.350498


## Updating the predictor

For this, we have two options:

### `BaseMixer.partial_fit()`

Updates a single mixer. You need to pass the new data wrapped in `EncodedDs` objects.

**Arguments:** 
* `train_data: EncodedDs`
* `dev_data: EncodedDs`

If the mixer does not need a `dev_data` partition, pass a dummy:

```
dev_data = EncodedDs(predictor.encoders, pd.DataFrame(), predictor.target)
```

### `PredictorInterface.adjust()`

Updates all mixers inside the predictor by calling their respective `partial_fit()` methods.

**Arguments:**
* `new_data: Union[EncodedDs, ConcatedEncodedDs, pd.DataFrame]`
* `old_data: Optional[Union[EncodedDs, ConcatedEncodedDs, pd.DataFrame]]`

Let's `adjust` our predictor:

In [5]:
from lightwood.data import EncodedDs

train_ds = EncodedDs(predictor.encoders, train_df, target)
update_ds = EncodedDs(predictor.encoders, update_df, target)

predictor.adjust(update_ds, train_ds)  # data to adjust and original data

INFO:lightwood-2278:Updating the mixers
/opt/hostedtoolcache/Python/3.9.12/x64/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
INFO:lightwood-2278:Loss @ epoch 1: 0.06666224698225658
DEBUG:lightwood-2278: `adjust` runtime: 5.34 seconds


In [6]:
new_predictions = predictor.predict(test_df)
new_predictions

INFO:lightwood-2278:Dropping features: []
INFO:lightwood-2278:Cleaning the data
INFO:lightwood-2278:Dropping features: []
DEBUG:lightwood-2278: `preprocess` runtime: 0.01 seconds
INFO:lightwood-2278:Featurizing the data
DEBUG:lightwood-2278: `featurize` runtime: 0.0 seconds
INFO:lightwood-2278:The block ICP is now running its explain() method
INFO:lightwood-2278:The block AccStats is now running its explain() method
INFO:lightwood-2278:AccStats.explain() has not been implemented, no modifications will be done to the data insights.
INFO:lightwood-2278:The block ConfStats is now running its explain() method
INFO:lightwood-2278:ConfStats.explain() has not been implemented, no modifications will be done to the data insights.
DEBUG:lightwood-2278: `predict` runtime: 1.77 seconds


,original_index,prediction,confidence,lower,upper
0,0,50.047745,0.9991,0.0,100.611655
1,1,19.834124,0.9991,0.0,70.398033
2,2,23.211976,0.9991,0.0,73.775885
3,3,21.163995,0.9991,0.0,71.727905
4,4,39.810138,0.9991,0.0,90.374048
...,...,...,...,...,...
201,201,48.718162,0.9991,0.0,99.282072
202,202,41.109587,0.9991,0.0,91.673497
203,203,39.365741,0.9991,0.0,89.929650
204,204,30.945220,0.9991,0.0,81.509130


Nice! Our predictor was updated, and new predictions are looking good. Let's compare the old and new accuracies:

In [7]:
from sklearn.metrics import r2_score
import numpy as np

old_acc = r2_score(test_df['concrete_strength'], predictions['prediction'])
new_acc = r2_score(test_df['concrete_strength'], new_predictions['prediction'])

print(f'Old Accuracy: {round(old_acc, 3)}\nNew Accuracy: {round(new_acc, 3)}')

Old Accuracy: 0.442
New Accuracy: 0.473


After updating, we see an increase in the R2 score of predictions for the held out test set.

## Conclusion

We have gone through a simple example of how Lightwood predictors can leverage newly acquired data to improve their predictions. The interface for doing so is fairly simple, requiring only some new data and a single call to update.

You can further customize the logic for updating your mixers by modifying the `partial_fit()` methods in them.